# サヤ取り
---

## A. クラッシュ（クラック）スプレッド

+ 原油と暖房油
+ 原油とガソリン
+ Quantopianで利用できる先物一覧 [available-futures](https://www.quantopian.com/help#available-futures)
+ いつもに比べてさやが大きくなったら戻るよ。という意味かな？

#### メモ
+ ```continuous_future``` と ```history``` の
    + 使い方
    + 使い分け
    + データ見比べ


## B. 限月間スプレッド

+ フォワードカーブ

## C. 異銘柄間

+ WTI と ブレント，ドバイ
+ 金と白金（質が違うので非推奨）



# 商品先物市場の分析
---

##1. 市場のコンディション
+ 順ザヤ（コンタンゴ）
+ 逆ザヤ（バックワーデーション）
    
##2. サヤの一代足の分析
+ カレンダースプレッド：期近の売り，期先の買い
+ 逆カレンダースプレッド：期近の買い，期先の売り
  

> システムトレード乗るかどうか、やや厄介な箇所があります。
56頁。商品先物は最後まで持てないので、どこかで乗り換えたり決済する必要があるということです。
54頁。限月があるので、FXや株のように単純に同じ銘柄を分析するということができないということです。
quantopianに適するかどうかご検討頂きますようお願い致します。
1個目の問題は、2番限のつなぎ足（continuos#2)で対応できるかもしれません。



In [ ]:
from quantopian.research.experimental import continuous_future, history
months = list("FGHJKMNQUVXZ")
dict_month = dict(zip(range( 1,13), months))


In [ ]:
## 原油と暖房油
Light_Sweet_Crude_Oil = "CL"
RBOB_Gasoline_Futures = "XB"
NY_Harbor_USLD_Futures = "HO"

ct_cl = continuous_future(Light_Sweet_Crude_Oil)
ct_xb = continuous_future(RBOB_Gasoline_Futures)
ct_ho = continuous_future(NY_Harbor_USLD_Futures)

start = "2017-1-1"
end="2018-5-13"
df = history(
 [ct_cl,ct_xb,ct_ho], 
 fields='price', 
 frequency='daily', 
 start=start, 
 end=end
)
df.columns = map(lambda x: x.root_symbol, df.columns)



In [ ]:
# 商品先物データへのアクセス

#continueous と hisotory のデータに違いがないか確認
#oilと ho の満期日がずれていないか確認

#clf = symbols("CL{}{}".format(dict_month[6], 18)) 
#xbf = symbols("XB{}{}".format(dict_month[6], 18)) 
#hof = symbols("HO{}{}".format(dict_month[6], 18))

clf17_june = symbols("XB{}{}".format(dict_month[6], 17)) 
clf18_june = symbols("XB{}{}".format(dict_month[6], 18)) 
clf17_december = symbols("XB{}{}".format(dict_month[12], 17)) 
clf18_december = symbols("XB{}{}".format(dict_month[12], 18)) 


clf_data = history(
 [clf17_june,clf18_june,clf17_december, clf18_december,] , 
 fields="price", ## コレを指定しない場合は，price だけSerieseで返す
 frequency='daily',
 start=start,
 end=end
 )


In [ ]:
df_change = clf_data.pct_change()

(df_change[clf17_june] - df_change[clf18_june]).dropna().cumsum().plot()
(df_change[clf17_december] - df_change[clf18_december]).dropna().cumsum().plot()

## data アクセス方法

+ symbols関数で，Futureはインスタンスを作って，データにアクセス
+ Symbol+月コード+年コード
+ 例 CLF160 CL=CrudeOil F=January 16=2016
+ デリバリが１月なのにexpirydateが２０１５年１２月２１日なのはなぜなんだ？


In [ ]:
clf16 = symbols('CLF16')
clf16

In [ ]:
dir(clf16)

+ root_symbol：原資産シンボル
+ start_date：Quantopian上で，データが利用可能な最初の日．start_dateの近辺ではデータがNaNの時がある．それは，実際取引が行われていないため．
+ end_date：受け渡し日直前の最後のトレード可能日
+ notice_date：受渡し日
+ auto_close_date：notice_dateもしくはend_dateのうち，先に来る日付から2日営業日前．今回はnotice_dateが12/23，end_dateが12/21なので，auto_close_dateは，end_dateの2営業日前12/17．バックテストでは，auto_close_dateで自動クローズ
+ tick_size：先物のきざみ
+ multiplier：単元

## クラッシュ
+ いつもよりさやが開いたらLongShortかな？
+ 正直わからない。あとで。


In [ ]:
df.pct_change().dropna().describe()

## カレンダースプレッド
+ offset を使う。


In [ ]:
f = 'SY'
f1 = continuous_future(f,  adjustment=None)
f2 = continuous_future(f, offset=1,  adjustment=None)
f3 = continuous_future(f, offset=2,  adjustment=None)
f4 = continuous_future(f, offset=3,  adjustment=None)
f5 = continuous_future(f, offset=4,  adjustment=None)

start = "2015-1-1"
end="2018-5-13"

f_pricing = history(
 [f1, f2, f3, f4, f5],
 fields='price', 
 frequency='daily', 
 start=start, 
 end=end
)
f_pricing.columns = map(lambda x: "{}_{}".format(x.root_symbol,x.offset), f_pricing.columns)


In [ ]:
## 満期日
sy_symbols = list()

for m in range(1,13):
    for y in [17, 18]:
        try:
            sy_symbols.append(symbols("SY{}{}".format(dict_month[m], y)))
        except:
            pass 


In [ ]:
for s in sy_symbols:
    print(s.expiration_date)

In [ ]:
df = f_pricing.loc["2018-4-17":"2018-5-12"].T
df.plot(marker="o")
# グリッドを合わせたいけど，どうしたらいいんだろう？


In [ ]:
df_change = f_pricing.pct_change(axis=1)
l = list()
for i, row in df_change.iterrows():
    lr = sp.stats.linregress(range(0,4), row[df_change.columns != 'SY_0'])
    l.append(lr.slope)
    
df_change['slope'] = l
df_change

    

In [ ]:
df_change[['slope', 'SY_4', 'SY_1']].plot()

In [ ]:
f_pricing.pct_change().plot(kind="scatter", x="SY_0", y="SY_5")

In [ ]:
import scipy as sp 
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(
    f_pricing["SY_0"].pct_change()[1:],
    f_pricing["SY_5"].pct_change()[1:],
)
intercept

In [ ]:
f_pricing["model_SY5"] = f_pricing["SY_0"]*(1+intercept)
f_pricing["diff"] = f_pricing["SY_5"] / f_pricing["model_SY5"]  - 1
f_pricing

In [ ]:
f_pricing[["SY_0", "SY_5", "diff"]].plot(secondary_y="diff")

In [ ]:
def my_regress(xs, ys):
    slope, intercept, r_value, p_value, std_err = sp.stats.linregress(xs,ys)
    return slope

f_pricing["SY_0_pct_change"] = f_pricing["SY_0"].pct_change()
f_pricing["SY_5_pct_change"] = f_pricing["SY_5"].pct_change()
        

In [ ]:
term = 14
import numpy as np
l = list()
for i in range(len(f_pricing)):
    if i >= term:
        xs = f_pricing.iloc[i-term:i]["SY_0_pct_change"]
        ys = f_pricing.iloc[i-term:i]["SY_5_pct_change"]
        l.append(my_regress(xs, ys))
    else:
        l.append(np.nan)
f_pricing["rolling 14 slope"]   = l

In [ ]:
f_pricing[["SY_0", "SY_5", "rolling 14 slope"]].plot(secondary_y="rolling 14 slope")

In [ ]:
np.random.seed(12345678)
x = np.random.random(10)
y = np.random.random(10)
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(x, y)
slope, intercept, r_value, p_value, std_err